In [5]:
# PyFEMM translation of the provided Octave geometry script
# Units: inches, planar
# Requires: pyFEMM (femm module)

import femm
import math
import time

pi = math.pi
sin = math.sin
cos = math.cos
asin = math.asin

def sind(x): return sin(math.radians(x))
def cosd(x): return cos(math.radians(x))
def tand(x): return math.tan(math.radians(x))

tic = time.perf_counter()

print('========================')
print(time.ctime())
print('========================')
print('Geometry')
print('========================')

# ----- Problem definition (same as Octave) -----
Length = 3.300  # motor length (in)
femm.openfemm()
femm.newdocument(0)
femm.mi_probdef(0, 'inches', 'planar', 1e-8, Length, 30, 0)

# ----- Stator geometry params (inches) -----
Nslots = 48
StatorOD = 10.600
StatorID = 6.375
ShoeHeight = 0.040  # Hs0
ShoeRadius = 0.020  # Hs1
SlotHeight = 1.149  # Hs2
SlotDia = 0.222     # 2*Rs
PostHeight = SlotDia/2 + SlotHeight + ShoeRadius + ShoeHeight
SlotOpen = 0.076    # Bs0
SlotWidthTop = 0.124
SlotWidthBot = SlotDia

StatorPitchAirgap = pi*StatorID/Nslots
StatorPitchSlotTop = pi*(StatorID + 2*ShoeHeight)/Nslots
StatorPitchSlotBot = pi*(StatorID + 2*ShoeHeight + 2*ShoeRadius + 2*SlotHeight)/Nslots
PostThickTop = StatorPitchSlotTop - SlotWidthTop
PostThickBot = StatorPitchSlotBot - SlotWidthBot

# ----- Rotor geometry params (inches) -----
Npoles = 8
RotorOD = 6.314
RotorID = 4.356
BridgeID = 0.287
DuctMinDia = RotorID + 2*BridgeID
DuctThick = 0.185
RibHeight = 0.118
RibWidth = 0.551
DistMinMag = 0.177
MagThick = 0.255
MagWidth = 0.715
Bridge = 0.056
DuctMaxDia = RotorOD - 2*Bridge
alpha = 66.5  # degrees
theta = 360/Npoles/2  # electrical/mechanical half-pole in degrees

# ----- Draw stator and rotor base circles/arcs -----
femm.mi_zoom(-StatorOD/2, -StatorOD/2, StatorOD/2, StatorOD/2)

# Stator OD
femm.mi_addnode(StatorOD/2, 0)
femm.mi_addnode(-StatorOD/2, 0)
femm.mi_addarc(StatorOD/2, 0, -StatorOD/2, 0, 180, 1)
femm.mi_addarc(-StatorOD/2, 0, StatorOD/2, 0, 180, 1)

# Rotor OD
femm.mi_addnode(RotorOD/2, 0)
femm.mi_addnode(-RotorOD/2, 0)
femm.mi_addarc(RotorOD/2, 0, -RotorOD/2, 0, 180, 1)
femm.mi_addarc(-RotorOD/2, 0, RotorOD/2, 0, 180, 1)

# Rotor ID
femm.mi_addnode(RotorID/2, 0)
femm.mi_addnode(-RotorID/2, 0)
femm.mi_addarc(RotorID/2, 0, -RotorID/2, 0, 180, 1)
femm.mi_addarc(-RotorID/2, 0, RotorID/2, 0, 180, 1)

# ----- One stator slot + tooth, then pattern copy -----
x1 = StatorID/2 * cos(asin((StatorPitchAirgap - SlotOpen)/StatorID))
y1 = (StatorPitchAirgap - SlotOpen)/2
x2 = x1
y2 = -y1

femm.mi_addnode(x1, y1)
femm.mi_addnode(x2, y2)
femm.mi_addarc(x2, y2, x1, y1, 2*asin((StatorPitchAirgap - SlotOpen)/StatorID)*180/pi, 1)

x3 = StatorID/2 + ShoeHeight
y3 = y1
femm.mi_addnode(x3, y3)
femm.mi_addsegment(x1, y1, x3, y3)

x4 = x3
y4 = y2
femm.mi_addnode(x4, y4)
femm.mi_addsegment(x2, y2, x4, y4)

x5 = StatorID/2 + ShoeHeight + ShoeRadius
y5 = (StatorPitchSlotTop - SlotWidthTop)/2
femm.mi_addnode(x5, y5)
femm.mi_addarc(x3, y3, x5, y5, 90, 1)

x6 = x5
y6 = -y5
femm.mi_addnode(x6, y6)
femm.mi_addarc(x6, y6, x4, y4, 90, 1)

x7 = StatorID/2 + ShoeHeight + ShoeRadius + SlotHeight
y7 = (StatorPitchSlotBot - SlotWidthBot)/2
femm.mi_addnode(x7, y7)
femm.mi_addsegment(x5, y5, x7, y7)

x8 = x7
y8 = -y7
femm.mi_addnode(x8, y8)
femm.mi_addsegment(x6, y6, x8, y8)

x9  = (StatorID/2 + ShoeHeight + ShoeRadius + SlotHeight)*cosd(360/Nslots) \
    + (StatorPitchSlotBot - SlotWidthBot)/2 * sind(360/Nslots)
y9  = (StatorID/2 + ShoeHeight + ShoeRadius + SlotHeight)*sind(360/Nslots) \
    - (StatorPitchSlotBot - SlotWidthBot)/2 * cosd(360/Nslots)
femm.mi_addnode(x9, y9)
femm.mi_addarc(x7, y7, x9, y9, 180, 1)

x10 = (StatorID/2 + ShoeHeight + ShoeRadius)*cosd(360/Nslots) \
    + (StatorPitchSlotTop - SlotWidthTop)/2 * sind(360/Nslots)
y10 = (StatorID/2 + ShoeHeight + ShoeRadius)*sind(360/Nslots) \
    - (StatorPitchSlotTop - SlotWidthTop)/2 * cosd(360/Nslots)
femm.mi_addnode(x10, y10)
femm.mi_addsegment(x5, y5, x10, y10)

# select primitive slot/tooth entities into group 1
femm.mi_selectarcsegment((x1+x2)/2, (y1+y2)/2)
femm.mi_selectsegment((x1+x3)/2, (y1+y3)/2)
femm.mi_selectsegment((x2+x4)/2, (y2+y4)/2)
femm.mi_selectarcsegment((x3+x5)/2, (y3+y5)/2)
femm.mi_selectarcsegment((x4+x6)/2, (y4+y6)/2)
femm.mi_selectsegment((x5+x7)/2, (y5+y7)/2)
femm.mi_selectsegment((x6+x8)/2, (y6+y8)/2)
femm.mi_selectarcsegment((x7+x9)/2, (y7+y9)/2)
femm.mi_selectsegment((x5+x10)/2, (y5+y10)/2)
femm.mi_setgroup(1)
femm.mi_selectgroup(1)
femm.mi_copyrotate2(0, 0, 360/Nslots, Nslots, 4)

# include stator OD arcs into same group
femm.mi_selectarcsegment(0, StatorOD/2)
femm.mi_selectarcsegment(0, -StatorOD/2)
femm.mi_setgroup(1)
femm.mi_clearselected()

# ----- Rotor features: duct + magnet construction for one pole sector -----
x11 = DuctMinDia/2
y11 = 0
x12 = x11 + DuctThick/sind(alpha)
y12 = 0
femm.mi_addnode(x11, y11)
femm.mi_addnode(x12, y12)

x13 = x12 + (DistMinMag/2)/tand(alpha)
y13 =  DistMinMag/2
femm.mi_addnode(x13, y13)
femm.mi_addsegment(x12, y12, x13, y13)

x14 = x13 - DuctThick*math.cos(math.radians(90-alpha))
y14 = y13 + DuctThick*math.sin(math.radians(90-alpha))
femm.mi_addnode(x14, y14)
femm.mi_addsegment(x11, y11, x14, y14)
femm.mi_addsegment(x13, y13, x14, y14)

x15 = x14 + MagWidth*cosd(alpha)
y15 = y14 + MagWidth*sind(alpha)
femm.mi_addnode(x15, y15)

x16 = x13 + MagWidth*cosd(alpha)
y16 = y13 + MagWidth*sind(alpha)
femm.mi_addnode(x16, y16)
femm.mi_addsegment(x15, y15, x16, y16)
femm.mi_addsegment(x13, y13, x16, y16)
femm.mi_addarc(x16, y16, x15, y15, 180, 1)

x17 = x13 - MagThick*math.cos(math.radians(90-alpha))
y17 = y13 + MagThick*math.sin(math.radians(90-alpha))
femm.mi_addnode(x17, y17)

x18 = x16 - MagThick*math.cos(math.radians(90-alpha))
y18 = y16 + MagThick*math.sin(math.radians(90-alpha))
femm.mi_addnode(x18, y18)

femm.mi_addsegment(x14, y14, x17, y17)
femm.mi_addsegment(x18, y18, x15, y15)
femm.mi_addsegment(x18, y18, x17, y17)

# Select this rotor feature set to group 2
femm.mi_selectsegment((x11+x14)/2, (y11+y14)/2)
femm.mi_selectsegment((x12+x13)/2, (y12+y13)/2)
femm.mi_selectsegment((x13+x14)/2, (y13+y14)/2)
femm.mi_selectsegment((x14+x17)/2, (y14+y17)/2)
femm.mi_selectsegment((x13+x16)/2, (y13+y16)/2)
femm.mi_selectsegment((x15+x16)/2, (y15+y16)/2)
femm.mi_selectsegment((x15+x18)/2, (y15+y18)/2)
femm.mi_selectarcsegment((x15+x16)/2, (y15+y16)/2)
femm.mi_selectsegment((x17+x18)/2, (y17+y18)/2)
femm.mi_setgroup(2)

# Mirror about x-axis, then rotate-copy around full rotor
femm.mi_selectgroup(2)
femm.mi_mirror2(0, 0, 1, 0, 4)  # mirror across x-axis
femm.mi_selectgroup(2)
femm.mi_copyrotate2(0, 0, 2*theta, Npoles, 4)

# Put rotor and bore arcs in same group
femm.mi_selectarcsegment(0, RotorOD/2)
femm.mi_selectarcsegment(0, -RotorOD/2)
femm.mi_selectarcsegment(0, RotorID/2)
femm.mi_selectarcsegment(0, -RotorID/2)
femm.mi_setgroup(2)
femm.mi_clearselected()

toc = time.perf_counter()
print(f'Done. Elapsed: {toc - tic:0.3f} s')


Tue Nov 11 17:08:51 2025
Geometry
Done. Elapsed: 0.952 s


In [6]:
# ----- Material assignments, boundaries, labels, mesh, solve -----

print('Material Assignments')

# Boundary
femm.mi_addboundprop('AirBound', 0,0,0,0,0,0,0,0,0)
femm.mi_selectarcsegment(0, StatorOD/2)
femm.mi_selectarcsegment(0,-StatorOD/2)
femm.mi_selectarcsegment(0, RotorID/2)
femm.mi_selectarcsegment(0,-RotorID/2)
femm.mi_setarcsegmentprop(5, 'AirBound', 0, 1)
femm.mi_clearselected()

# Built-in materials
femm.mi_getmaterial('Air')
femm.mi_getmaterial('20 AWG')

# Custom materials
# N36Z_20 magnet (Br ≈ 0.667 T, μr ≈ 1.03, σ = 920000 S/m)
femm.mi_addmaterial('N36Z_20', 1.03, 1.03, 920000, 0, 0.667)

# M19_29G steel with B–H curve and lamination data (lam thickness 0.34 mm, fill 0.94, density 7.65e3? -> FEMM uses ρ as ρmass? Here we set typical lam data per your call)
# mi_addmaterial(name, mux, muy, Hc, J, Cduct, Lam_d, Phi_hmax, LamFill, LamType)
femm.mi_addmaterial('M19_29G', 0, 0, 0, 0, 1.9, 0.34, 0, 0.94, 0)

# BH curve points
BH_B = [0,0.05,0.1,0.15,0.36,0.54,0.65,0.99,1.2,1.28,1.33,1.36,1.44,1.52,
        1.58,1.63,1.67,1.8,1.9,2,2.1,2.3,2.5,2.563994494,3.779889874]
BH_H = [0,22.28,25.46,31.83,47.74,63.66,79.57,159.15,318.3,477.46,636.61,
        795.77,1591.5,3183,4774.6,6366.1,7957.7,15915,31830,111407,190984,
        350135,509252,560177.2,1527756]
for B,H in zip(BH_B, BH_H):
    femm.mi_addbhpoint('M19_29G', B, H)

# Outside stator region -> <No Mesh>
femm.mi_addblocklabel(0, 0.6*StatorOD)
femm.mi_selectlabel(0, 0.6*StatorOD)
femm.mi_setblockprop('<No Mesh>', 1, 0, 'None', 0, 0, 0)
femm.mi_clearselected()

# Stator steel
femm.mi_addblocklabel(0, 0.5*(StatorOD/2 + x7))
femm.mi_selectlabel(0, 0.5*(StatorOD/2 + x7))
femm.mi_setblockprop('M19_29G', 1, 0, 'None', 0, 1, 0)
femm.mi_clearselected()

# Airgap annulus
femm.mi_addblocklabel(0, 0.25*(StatorID + RotorOD))
femm.mi_selectlabel(0, 0.25*(StatorID + RotorOD))
femm.mi_setblockprop('Air', 1, 0, 'None', 0, 0, 0)
femm.mi_clearselected()

# Rotor steel
femm.mi_addblocklabel(0, 0.5*(RotorOD/2 + x12))
femm.mi_selectlabel(0, 0.5*(RotorOD/2 + x12))
femm.mi_setblockprop('M19_29G', 1, 0, 'None', 0, 2, 0)
femm.mi_clearselected()

# Rotor ducts and magnet air pockets (one pole), then pattern
femm.mi_addblocklabel(0, 0.5*(x11 + x12))
femm.mi_selectlabel(0, 0.5*(x11 + x12))
femm.mi_setblockprop('Air', 1, 0, 'None', 0, 2, 0)

lbl_x = 0.5*(x15 + x16) + DuctThick/4*cosd(alpha)
lbl_y = 0.5*(y15 + y16) + DuctThick/4*sind(alpha)
femm.mi_addblocklabel(lbl_x,  lbl_y)
femm.mi_selectlabel(lbl_x,  lbl_y)
femm.mi_setblockprop('Air', 1, 0, 'None', 0, 2, 0)

femm.mi_addblocklabel(lbl_x, -lbl_y)
femm.mi_selectlabel(lbl_x, -lbl_y)
femm.mi_setblockprop('Air', 1, 0, 'None', 0, 2, 0)

# copy those three labels around all poles
femm.mi_copyrotate2(0, 0, 2*theta, Npoles, 2)
femm.mi_clearselected()

# Magnet blocks for one quadrant, then rotate pattern per script
mx = 0.25*(x13 + x17 + x16 + x18)
my = 0.25*(y13 + y17 + y16 + y18)

# +Y, north
femm.mi_addblocklabel(mx,  my)
femm.mi_selectlabel(mx,  my)
femm.mi_setblockprop('N36Z_20', 1, 0, 'None', -90 + alpha, 3, 0)
femm.mi_moverotate(0, 0, 2*theta)
femm.mi_clearselected()

# -Y, south
femm.mi_addblocklabel(mx, -my)
femm.mi_selectlabel(mx, -my)
femm.mi_setblockprop('N36Z_20', 1, 0, 'None',  90 - alpha, 3, 0)
femm.mi_moverotate(0, 0, 2*theta)
femm.mi_clearselected()

# +Y, opposite polarity
femm.mi_addblocklabel(mx,  my)
femm.mi_selectlabel(mx,  my)
femm.mi_setblockprop('N36Z_20', 1, 0, 'None', 180 - 90 + alpha, 3, 0)
femm.mi_clearselected()

# -Y, opposite polarity
femm.mi_addblocklabel(mx, -my)
femm.mi_selectlabel(mx, -my)
femm.mi_setblockprop('N36Z_20', 1, 0, 'None', 180 + 90 - alpha, 3, 0)
femm.mi_clearselected()

# replicate magnet labels: select group 3 then copy-rotate by 4*theta, Npoles/2 times
femm.mi_selectgroup(3)
femm.mi_copyrotate(0, 0, 4*theta, int(Npoles/2))
femm.mi_clearselected()

# Inner bore region -> <No Mesh>
femm.mi_addblocklabel(0, 0.25*RotorID)
femm.mi_selectlabel(0, 0.25*RotorID)
femm.mi_setblockprop('<No Mesh>', 1, 0, 'None', 0, 0, 0)
femm.mi_clearselected()

# Simple coil placeholders in stator slot near mid-tooth, rotated across all slots
cx = 0.5*(x5 + x7)*cosd(360/Nslots/2)
cy = 0.5*(x5 + x7)*sind(360/Nslots/2)
femm.mi_addblocklabel(cx, cy)
femm.mi_selectlabel(cx, cy)
femm.mi_setblockprop('20 AWG', 1, 0, 'None', 0, 1, 1)
femm.mi_copyrotate(0, 0, 360/Nslots, Nslots)
femm.mi_clearselected()

# --- Save, mesh, solve, and postprocessing ---

femm.mi_saveas('ToyotaPrius.FEM')
femm.mi_clearselected()
femm.mi_createmesh()
femm.mi_analyze(0)
femm.mi_loadsolution()
femm.mo_showdensityplot(1, 0, 2.0, 0.0, 'mag')
femm.mo_setgrid(0.5, 'cart')
femm.mo_showvectorplot(1, 1)

print('========================')
print(time.ctime())
print('========================')


Material Assignments
Tue Nov 11 17:12:59 2025
